In [4]:
from pathlib import Path
import sys
parent = Path.cwd().resolve().parents[0]
sys.path.append(str(parent))
from MomentumBacktestClass import MomentumVectorBacktest as M
import pandas as pd
import requests
import re

In [ ]:
start_date = '2022-01-01'
end_date = '2025-01-01'

# We need to ensure that, for whatever backtest window we choose, price data is available for every stock so the performance comparison is fair and measured over the same exact period. 
# Therefore, the earliest valid start date should be the latest common date for which data for all 101 assets of the Nasday 100 are available.

# Getting the yahoo finance tickers for all stocks in the Nasdaq 100

In [3]:
url = "https://en.wikipedia.org/wiki/Nasdaq-100"

headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 "
                  "(KHTML, like Gecko) Chrome/120.0 Safari/537.36"
}

html = requests.get(url, headers=headers, timeout=30).text
tables = pd.read_html(html)

components = next(df for df in tables if "Ticker" in df.columns)
tickers = components["Ticker"].tolist()

# Yahoo Finance uses '-' instead of '.' for tickers like BRK.B -> BRK-B
tickers = [t.replace(".", "-") for t in tickers]

print(len(tickers))
print(tickers[:15])

101
['ADBE', 'AMD', 'ABNB', 'GOOGL', 'GOOG', 'AMZN', 'AEP', 'AMGN', 'ADI', 'AAPL', 'AMAT', 'APP', 'ARM', 'ASML', 'AZN']


C:\Users\luciu\AppData\Local\Temp\ipykernel_14500\317895948.py:9: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  tables = pd.read_html(html)


In [7]:
bt = M('AAPL',momentum=10,start=start_date,end=end_date,tc=0.0)
bt.run_strategy()

[*********************100%***********************]  1 of 1 completed


'buy&hold: 49.90%, strategy: 71.35%, alpha: 21.45%'

In [13]:
summary_dict = {}
for ticker in tickers: 
    bt = M(ticker,momentum=10,start=start_date,end=end_date,tc=0.0)
    summary_dict[ticker] = bt.run_strategy()

print(summary_dict)

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%********

{'ADBE': 'buy&hold: -13.38%, strategy: 15.34%, alpha: 28.71%', 'AMD': 'buy&hold: -8.44%, strategy: 40.30%, alpha: 48.74%', 'ABNB': 'buy&hold: -15.05%, strategy: 126.74%, alpha: 141.79%', 'GOOGL': 'buy&hold: 39.70%, strategy: -10.28%, alpha: -49.98%', 'GOOG': 'buy&hold: 40.23%, strategy: -13.59%, alpha: -53.82%', 'AMZN': 'buy&hold: 38.05%, strategy: 35.10%, alpha: -2.95%', 'AEP': 'buy&hold: 14.39%, strategy: -1.65%, alpha: -16.04%', 'AMGN': 'buy&hold: 22.57%, strategy: 27.34%, alpha: 4.78%', 'ADI': 'buy&hold: 34.74%, strategy: -71.32%, alpha: -106.06%', 'AAPL': 'buy&hold: 49.90%, strategy: 71.35%, alpha: 21.45%', 'AMAT': 'buy&hold: 9.58%, strategy: -17.74%, alpha: -27.32%', 'APP': 'buy&hold: 347.03%, strategy: 323.11%, alpha: -23.93%', 'ARM': 'buy&hold: 122.31%, strategy: 80.38%, alpha: -41.93%', 'ASML': 'buy&hold: -0.13%, strategy: 42.79%, alpha: 42.93%', 'AZN': 'buy&hold: 16.94%, strategy: 3.95%, alpha: -12.99%', 'TEAM': 'buy&hold: -17.11%, strategy: 106.94%, alpha: 124.05%', 'ADSK': 

In [9]:
def pct_to_float(s: str) -> float:
    """
    Convert strings into decimals:
    """
    s = s.strip().replace(",", "").replace("%", "")
    return float(s) / 100.0

def results_dict_to_df(results: dict) -> pd.DataFrame:
    """
    returns DataFrame with columns bh, strat, alpha (decimals), bh_pct, strat_pct, alpha_pct
    """
    rows = []
    pat = re.compile(r"buy&hold:\s*([-0-9.,]+%)\s*,\s*strategy:\s*([-0-9.,]+%)\s*,\s*alpha:\s*([-0-9.,]+%)")

    for ticker, text in results.items():
        m = pat.search(text)
        if not m:
            raise ValueError(f"Could not parse line for {ticker}: {text}")

        bh_s, strat_s, alpha_s = m.groups()
        bh = pct_to_float(bh_s)
        strat = pct_to_float(strat_s)
        alpha = pct_to_float(alpha_s)

        rows.append({
            "ticker": ticker,
            "bh": bh,
            "strategy": strat,
            "alpha": alpha,
            "bh_pct": bh * 100,
            "strategy_pct": strat * 100,
            "alpha_pct": alpha * 100,
        })

    df = pd.DataFrame(rows).set_index("ticker")
    return df

def top_n(df: pd.DataFrame, rank_by: str = "alpha", n: int = 20, ascending: bool = False) -> pd.DataFrame:
    """
    rank_by: 'bh', 'strategy', or 'alpha'
    """
    if rank_by not in {"bh", "strategy", "alpha"}:
        raise ValueError("rank_by must be one of: 'bh', 'strategy', 'alpha'")

    out = df.sort_values(rank_by, ascending=ascending).head(n).copy()

    # Pretty formatting for display
    out["buy&hold"] = out["bh"].map(lambda x: f"{x:.2%}")
    out["strategy_ret"] = out["strategy"].map(lambda x: f"{x:.2%}")
    out["alpha_ret"] = out["alpha"].map(lambda x: f"{x:.2%}")

    return out[["buy&hold", "strategy_ret", "alpha_ret"]]

In [14]:
df = results_dict_to_df(summary_dict)

# Top 20 by Total return

In [18]:
print('Top 20 by buy&hold (Total return of asset over period)')
print(top_n(df, rank_by="bh", n=20))

Top 20 by buy&hold (Total return of asset over period)
       buy&hold strategy_ret alpha_ret
ticker                                
MSTR    518.48%     3458.19%  2939.72%
NVDA    419.30%      311.95%  -107.35%
PLTR    405.21%      -32.33%  -437.54%
AXON    349.56%       35.16%  -314.40%
APP     347.03%      323.11%   -23.93%
CEG     334.03%       10.69%  -323.34%
AVGO    329.29%      -42.91%  -372.20%
ARM     122.31%       80.38%   -41.93%
PANW    117.32%      -43.70%  -161.02%
TMUS    110.42%      -44.49%  -154.91%
BKNG    110.24%       69.16%   -41.07%
CRWD     97.95%      -43.96%  -141.91%
COST     95.89%       56.50%   -39.38%
CDNS     95.70%      -65.44%  -161.15%
CTAS     92.32%      -24.15%  -116.47%
PCAR     84.82%       14.38%   -70.45%
META     84.74%      -61.89%  -146.63%
MAR      79.58%      -23.98%  -103.55%
ORLY     78.14%       11.81%   -66.32%
ISRG     76.64%      -44.75%  -121.39%


# Top 20 by Strategy Return

In [ ]:
print('Top 20 by strategy return')
print(top_n(df, rank_by="strategy", n=20)) 

Top 20 by strategy return
       buy&hold strategy_ret alpha_ret
ticker                                
MSTR    518.48%     3458.19%  2939.72%
APP     347.03%      323.11%   -23.93%
TSLA     17.57%      319.96%   302.40%
NVDA    419.30%      311.95%  -107.35%
ABNB    -15.05%      126.74%   141.79%
TEAM    -17.11%      106.94%   124.05%
ARM     122.31%       80.38%   -41.93%
AAPL     49.90%       71.35%    21.45%
BKNG    110.24%       69.16%   -41.07%
NFLX     74.49%       61.44%   -13.06%
COST     95.89%       56.50%   -39.38%
ASML     -0.13%       42.79%    42.93%
FAST     33.16%       42.72%     9.56%
AMD      -8.44%       40.30%    48.74%
CHTR    -41.18%       36.44%    77.62%
AXON    349.56%       35.16%  -314.40%
AMZN     38.05%       35.10%    -2.95%
CPRT     72.68%       34.42%   -38.26%
AMGN     22.57%       27.34%     4.78%
KHC      -6.32%       20.96%    27.28%


# Top 20 by Alpha (outperformance)

In [17]:
print('Top 20 by alpha (outperformance of gross return by strategy return)')
print(top_n(df, rank_by="alpha", n=20)) 

Top 20 by alpha (outperformance of gross return by strategy return)
       buy&hold strategy_ret alpha_ret
ticker                                
MSTR    518.48%     3458.19%  2939.72%
TSLA     17.57%      319.96%   302.40%
ABNB    -15.05%      126.74%   141.79%
TEAM    -17.11%      106.94%   124.05%
CHTR    -41.18%       36.44%    77.62%
INTC    -60.52%       13.94%    74.47%
AMD      -8.44%       40.30%    48.74%
ASML     -0.13%       42.79%    42.93%
ADBE    -13.38%       15.34%    28.71%
KHC      -6.32%       20.96%    27.28%
BIIB    -34.60%       -7.71%    26.89%
AAPL     49.90%       71.35%    21.45%
ZS      -30.40%      -11.19%    19.21%
MDLZ     -4.61%        7.64%    12.25%
MU       -7.65%        3.77%    11.43%
FAST     33.16%       42.72%     9.56%
EXC       4.88%       12.69%     7.81%
PYPL    -51.08%      -44.70%     6.38%
AMGN     22.57%       27.34%     4.78%
REGN     16.54%       17.54%     1.00%
